In [2]:
import dask.dataframe as dd
import sys
import time
import pandas as pd
import datetime
from fbprophet import Prophet
from matplotlib import pyplot as plt

In [ ]:
sys.getdefaultencoding() 

In [ ]:
filename = '/Users/bluekidds/Downloads/tn_data/IISIRoadMOE.txt'
filename_Bus = '/Users/bluekidds/Downloads/tn_data/IISIBusData.txt'

In [ ]:
!tail -15 /Users/bluekidds/Downloads/tn_data/IISIRoadMOE.txt

In [ ]:
!head -15 /Users/bluekidds/Downloads/tn_data/IISIBusData.txt

In [ ]:
df = dd.read_csv(filename, encoding="big5", header=None, sep=r"\s*", error_bad_lines=False, usecols=[1,3,4,7])
display(df.head())

In [ ]:
df = dd.read_csv(filename, encoding="big5", header=None, sep='\t', error_bad_lines=False, usecols=[1,3,6])
display(df.head())

In [ ]:
df_road = dd.read_csv(filename, encoding="big5", header=None, sep='\t', error_bad_lines=False, usecols=[1,3,6])

In [ ]:
df_road.astype({1: 'str'})
df_road.head()

In [ ]:
df_road[1].value_counts()

In [ ]:
df['time'] = df[[3,4]].apply(lambda x: ' '.join(x), axis = 1)
display(df.head())

In [ ]:
df = df.drop([3,4], axis=1)
display(df.head())

In [ ]:
columns = ['RoadID','time', 'speed']

df.columns = columns

display(df.head())

In [ ]:
df['time'] = pd.to_datetime(df['time'])
display(df.head())

In [ ]:
df1 = df[df['RoadID'] =='L03072']
df1 = df1.to_csv('data/*.csv')

In [ ]:
df_refine = df[['RoadID','time', 'speed']]
df_refine.head()

In [ ]:
df_refine1 = df_refine[df_refine['RoadID']=='L00620']
df_refine1 = df_refine1.compute()

In [ ]:
df_refine.to_csv('refine_data.csv')

In [ ]:
df1 = df[df['RoadName']=='中華南路二段(金華路一段>德興路)']
df1.head(10)

In [ ]:
df2 = df1.compute()

In [ ]:
df1 = df1[['time', 'speed']]
display(df1.tail(5))

In [ ]:
df.RoadName.value_counts().compute()

In [ ]:
def blocks(files, size=65536):
    while True:
        b = files.read(size)
        if not b: break
        yield b

with open('/Users/bluekidds/Downloads/tn_data/IISIRoadMOE.txt',"r", errors='replace') as f:
    print(sum(bl.count("\n") for bl in blocks(f)))

In [ ]:
!pip install pyarrow

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa

In [ ]:
input_file = open('/Users/bluekidds/Downloads/tn_data/IISIRoadMOE.txt', errors='replace')
#output_file = open('output.txt','w')
place_list = []
for lines in range(20):
    line = input_file.readline()
    
    line_list = line.split()
    
    place_list.append(','.join([line_list[0], line_list[1],line_list[3]+' '+line_list[4], line_list[7]]))
    #print(" ".join(line.split()))

    #print(line.split(), len(line.split()))
    #output_file.write(line)

small_df = pd.DataFrame(place_list)
table = pa.Table.from_pandas(small_df)
#with open('IISIRoadMOE_Processed.txt', 'w') as filehandle:
    #filehandle.writelines("%s\n" % place for place in place_list)
pq.write_table(table, 'example.parquet')    
#with open('/Users/bluekidds/Downloads/tn_data/IISIRoadMOE.txt', errors='replace') as f:
#    full_lines = f.readlines()

# parse full lines to get the text to right of "-"
#sample_num = 10
input_file.close()

#for i in range(sample_num):
#    print(" ".join(full_lines[i].split()))

In [ ]:
new_df = pd.read_parquet('example.parquet')
display(new_df.head())

In [ ]:
def istimecheck(time):
    timeformat = "%Y-%m-%d %H:%M:%S.%f"
    
    try:
        validtime = datetime.datetime.strptime(time, timeformat)
        return True
        #Do your logic with validtime, which is a valid format
    except ValueError:
        return False
        #Do your logic for invalid format (maybe print some message?).

In [ ]:
import logging

#logging.basicConfig(level=logging.info, file='errors.log')
logging.basicConfig(filename='errors.log',level=logging.DEBUG)

In [ ]:
def ProcessLargeTextFile():
    logging.basicConfig(filename='errors.log',level=logging.DEBUG)
    bunchsize = 1000000     # Experiment with different sizes
    bunch = []
    start = time.time()
    errors = 0
    time_errors = 0
    speed_errors = 0
    with open('/Users/bluekidds/Downloads/tn_data/IISIRoadMOE.txt', errors='replace') as r, open("IISIRoadMOE_Processed.txt", "w") as w:
        for line in r:
            line_list = line.split()
            if len(line_list) < 8:
                print('Error occur...')
                print(line_list)
                break
            
            my_time = line_list[3]+' '+line_list[4]
            

            if not istimecheck(my_time):
                #print('Error detected when extracting time')
                errors += 1
                time_errors += 1
                logging.error("Time format error:" + my_time)
                continue 
            if not line_list[5].replace('.','',1).isdigit():
                if float(line_list[7]) == -1:
                    pass
                else:
                    #print('Error detected when extracting speed')
                    errors += 1
                    speed_errors +=1
                    logging.error("Speed format error:" + line_list[5])
                    continue
                
            bunch.append(','.join([line_list[0], line_list[1],line_list[3]+' '+line_list[4], line_list[5] + '\n']))
            
            if len(bunch) == bunchsize:
                print('Processed 1 million lines in: ' + str(time.time()-start))
                print('Number of time errors accumulated: ' + str(time_errors))
                print('Number of speed errors acucmulated: ' + str(speed_errors))
                start = time.time()
                w.writelines(bunch)
                
                bunch = []
        w.writelines(bunch)
        print('Total errors: ' + str(errors) + ' and Speed variable errors ' + str(speed_errors) + ' and Time variable errors: ' + str(time_errors))

In [ ]:
ProcessLargeTextFile()

In [ ]:
'15.63157894�368419'.replace('.','',1).isdigit()

## Pipeline after processing data

In [ ]:
!tail -15 IISIRoadMOE_Processed.txt

In [ ]:
filename = 'IISIRoadMOE_Processed.txt'

df = dd.read_csv(filename, header=None)
display(df.head())

In [ ]:
filename = 'IISIRoadMOE_Processed.txt'

df = dd.read_csv(filename, header=None)
display(df.head())

In [ ]:
df = dd.read_csv(filename, header=None, error_bad_lines=False, usecols=[1,2,3])
columns = ['RoadID','time', 'speed']
df.columns = columns
df['time']=dd.to_datetime(df['time'])
display(df.head(10))

In [ ]:
road_list = df['RoadID'].value_counts().compute()

In [ ]:
road_ID_set = set(df['RoadID'])

For these road lists, if the number of values is greater than 50000. use it.

For each road, predict its last 2400 units using the past two months.

1. Set each df
2. Resample to 15 mins
3. df to fbpro ['2019-03-01 00:00:00' : '2019-05-02 00:00:00']
4. create_model(config)
5. future = model.make_future_dataframe(periods=2400, freq='15T')
6. forecast = model.predict(future)
7. forecast[y_hat] ['2019-05-02 00:00:00:2019-05-29 00:00:00']

In [ ]:
def create_model():
    
    
    
    model = Prophet(
        daily_seasonality=False,
        weekly_seasonality=False,
        yearly_seasonality=False,
        changepoint_prior_scale=0.02,
    )
    model.add_seasonality(
        name='daily',
        period=1,
        fourier_order=16,
        prior_scale=0.1,

    )
    model.add_seasonality(
        name='weekly',
        period=7,
        fourier_order=2,
        prior_scale=0.1,
    )
    return model

In [ ]:
road_list_effective = list(road_list[road_list > 30000].keys())
print(road_list_effective)

In [ ]:
def process_all_sections(df, road_list_effective):
    logging.basicConfig(filename='process_road_data.log',level=logging.DEBUG)
    start = time.time()
    for road_ID in road_list_effective:
        logging.info('Processing road: %s', road_ID)
        
        df_local = df[df['RoadID']== road_ID][['time','speed']]
        df_local= df_local.set_index('time')
        #df_local = df_local.compute()
        #if df_local.shape[0] < 30000:
        #    logging.error("Data size insufficient in road : " + road_ID + " " + str(df_local.shape[0]))
        #    now = time.time()
        #    print('Processed one road in: ' + str(now-start))
        #    start = now
        #    continue
        df_resample = df_local.resample('15min').mean().compute()
        
        fb_df = df_to_fbpro(df_resample['2019-03-01 05:30:00' : '2019-05-01 05:30:00'])
        
        
        
        model = create_model()
        model.fit(fb_df)
        future = model.make_future_dataframe(periods=2400, freq='15T')
        forecast = model.predict(future)
        forecast[['ds', 'yhat']].to_json('forecast_output/' + '.'.join((road_ID, 'json')))
        print('Finish ' + road_ID)
        now = time.time()
        print('Processed one road in: ' + str(now-start) + 's')
        start = now

In [ ]:
%%time
df_L00770 = df[df['RoadID']== 'L00770'][['time','speed']]
df_L00770= df_L00770.set_index('time')
df_resample = df_L00770.resample('15min').mean()

display(df_resample.head())

In [ ]:
process_all_sections(df, road_list_effective)

In [ ]:
df.head(10)['time']

In [ ]:
#df1 = df[df['RoadID']=='L03072'][['time','speed']]

df4 = df[df['RoadID']=='L00466'][['time','speed']]
df4= df4.set_index('time')  

#df1['time']=dd.to_datetime(df1['time'])
#display(df1.head(10))



In [ ]:
df4.c.shape

In [ ]:
display(df1.loc['2019-03-15 00:00:00' : '2019-03-17 10:21:00'].compute())

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
import plotly
import plotly.graph_objs as go
init_notebook_mode(connected=False)


In [ ]:
#df1 = df1.compute()
df4 = df4.compute()

In [ ]:


#df3 = df1.resample('15min').mean()
df5 = df4.resample('15min').mean()

display(df5.head())

In [ ]:
trace_fitted = go.Scatter(
              x=df5.index,
              y=df5.speed,
              name = 'actuals')

data = [trace_fitted]

layout = go.Layout(
    title='Actual Time Series Data in L00466',
    xaxis=dict(title='Date'))
fig = go.Figure(data=data, layout=layout)
    
iplot(fig)

In [ ]:
trace_fitted = go.Scatter(
              x=df3.index,
              y=df3.speed,
              name = 'actuals')

data = [trace_fitted]

layout = go.Layout(
    title='Actual Time Series Data in L03072',
    xaxis=dict(title='Date'))
fig = go.Figure(data=data, layout=layout)
    
iplot(fig)

In [ ]:
def df_to_fbpro(df):
    if len(df.columns) == 1:
        df = df.reset_index()
    df.columns = ['ds', 'y']
    return df
        

In [ ]:
fb_df = df_to_fbpro(df5['2019-03-01 05:30:00' : '2019-05-01 05:30:00'])
display(fb_df.head())

In [ ]:
fb_df_test = df_to_fbpro(df5['2019-05-01 05:30:00' : '2019-06-01 05:30:00'])
display(fb_df_test.head())

In [ ]:
df2.columns

In [ ]:
m = Prophet()
m.fit(fb_df)


In [ ]:
model = Prophet(
    daily_seasonality=False,
    weekly_seasonality=False,
    yearly_seasonality=False,
    changepoint_prior_scale=0.02,
)
model.add_seasonality(
    name='daily',
    period=1,
    fourier_order=16,
    prior_scale=0.1,
  
)
model.add_seasonality(
    name='weekly',
    period=7,
    fourier_order=2,
    prior_scale=0.1,
)


model.fit(fb_df);

In [ ]:
fb_df.shape

In [ ]:
#future = m.make_future_dataframe(periods=600, freq='H')
#future.tail()

future = model.make_future_dataframe(periods=2400, freq='15T')
future.tail()

In [ ]:
fb_df_test.tail()

In [ ]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


In [ ]:
fig2 = model.plot_components(forecast)


In [ ]:
fig1 = model.plot(forecast)

In [ ]:
forecast

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
ax.plot(fb_df['ds'], fb_df['y'], c='black', marker='o', ms=3, linestyle='None', label='Train')
ax.plot(fb_df_test['ds'], fb_df_test['y'], c='r', marker='o', ms=3, linestyle='None', label='Test')
ax.plot(forecast['ds'], forecast['yhat'], c='b', marker='o', ms=3, linestyle='None', label='Forecast', alpha=0.5)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Speed');

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
ax.plot(fb_df['ds'], fb_df['y'], c='black', marker='o', ms=3, linestyle='None', label='Train')
ax.plot(fb_df_test['ds'], fb_df_test['y'], c='r', marker='o', ms=3, linestyle='None', label='Test')
ax.plot(forecast['ds'], forecast['yhat'], c='b', marker='o', ms=3, linestyle='None', label='Forecast', alpha=0.5)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Speed');

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
ax.plot(fb_df['ds'], fb_df['y'], c='black', marker='o', ms=3, linestyle='None', label='Train')
ax.plot(fb_df_test['ds'], fb_df_test['y'], c='r', marker='o', ms=3, linestyle='None', label='Test')
ax.plot(forecast['ds'], forecast['yhat'], c='b', marker='o', ms=3, linestyle='None', label='Forecast', alpha=0.5)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Speed');

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
ax.plot(fb_df['ds'], fb_df['y'], c='black', marker='o', ms=3, linestyle='None', label='Train')
ax.plot(fb_df_test['ds'], fb_df_test['y'], c='r', marker='o', ms=3, linestyle='None', label='Test')
ax.plot(forecast['ds'], forecast['yhat'], c='b', marker='o', ms=3, linestyle='None', label='Forecast', alpha=0.5)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Speed');

## Cross-Validation

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(model, initial='40 days', period='3 days', horizon = '10 days')
df_cv.head()

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='rmse')


## Add holiday effects(Not supporting Taiwan)

In [3]:
import folium
import folium.plugins as plugins
import numpy as np

#m.fit(df)

In [ ]:
!pip install folium

In [ ]:
import folium
import folium.plugins as plugins
import numpy as np

np.random.seed(3141592)
initial_data = (
    np.random.normal(size=(100, 2)) * np.array([[1, 1]]) +
    np.array([[48, 5]])
)

move_data = np.random.normal(size=(100, 2)) * 0.01

data = [(initial_data + move_data * i).tolist() for i in range(100)]

weight = 1  # default value
for time_entry in data:
    for row in time_entry:
        row.append(weight)

In [ ]:
m = folium.Map(
    location=[35.68159659061569, 139.76451516151428],
    zoom_start=16
)

# Lon, Lat order.
lines = [
    {
        'coordinates': [
            [139.76451516151428, 35.68159659061569],
            [139.75964426994324, 35.682590062684206],
        ],
        'dates': [
            '2017-06-02T00:00:00',
            '2017-06-02T00:10:00'
        ],
        'color': 'red'
    },
    {
        'coordinates': [
            [139.75964426994324, 35.682590062684206],
            [139.7575843334198, 35.679505030038506],
        ],
        'dates': [
            '2017-06-02T00:10:00',
            '2017-06-02T00:20:00'
        ],
        'color': 'blue'
    },
    {
        'coordinates': [
            [139.7575843334198, 35.679505030038506],
            [139.76337790489197, 35.678040905014065],
        ],
        'dates': [
            '2017-06-02T00:20:00',
            '2017-06-02T00:30:00'
        ],
        'color': 'green',
        'weight': 15,
    },
    {
        'coordinates': [
            [139.76337790489197, 35.678040905014065],
            [139.76451516151428, 35.68159659061569],
        ],
        'dates': [
            '2017-06-02T00:30:00',
            '2017-06-02T00:40:00'
        ],
        'color': '#FFFFFF',
    },
]

features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
        },
        'properties': {
            'times': line['dates'],
            'style': {
                'color': line['color'],
                'weight': line['weight'] if 'weight' in line else 5
            }
        }
    }
    for line in lines
]

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M', add_last_point=True).add_to(m)
m

In [2]:
from datetime import datetime, timedelta


time_index = [
    (datetime.now() + k * timedelta(1)).strftime('%Y-%m-%d') for
    k in range(len(data))
]

NameError: name 'data' is not defined

* Time_index is a list of datetime objects
* data is a list of list of coordinates

In [ ]:
data[1][0]

In [ ]:
display(len(data), len(data[0]))

## Visualize Traffic Data

In [3]:
from glob import glob

file_paths = glob('forecast_output/*.json')


In [10]:
forecast_df = pd.read_json(file_paths[0])
location_id = file_paths[0].split('/')[1].split('.')[0]
forecast_df.columns = ['time',location_id]
forecast_df['time'] = forecast_df['time'] / 1000
forecast_df.set_index('time', inplace=True)
for file_path in file_paths[1:]:
    location_name = file_path.split('/')[1].split('.')[0]
    local_df = pd.read_json(file_path)
    local_df.columns = ['time',location_name]
    local_df['time'] = local_df['time'] / 1000
    local_df.set_index('time', inplace=True)
    #display(local_df.head())
    #local_df.index = pd.to_datetime(local_df.index)
    forecast_df = pd.merge(forecast_df, local_df, how='outer', on='time')
    #forecast_df = forecast_df.join(local_df, how='outer')
    #print(location_name, file_path)
#file_paths[0].split('/')[1].split('.')[0]


display(forecast_df.head())

,L04891,L09870,L01543,L06286,L01144,L05107,L05042,L01294,L00190,L02284,L01152,L00082,L01693,L03587,L01312,...,L05811,L06913,L04241,L05145,L04304,L01556,L00500,L98066,L07416,L04152,L03382,L04017,L04294,L05893,L05201
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.551420e+09,35.134963,40.085713,NaN,NaN,37.840989,NaN,40.093677,39.205644,30.599845,39.383541,NaN,NaN,NaN,NaN,NaN,...,NaN,31.488935,31.642293,29.726891,35.321547,NaN,35.607722,NaN,NaN,NaN,NaN,NaN,33.862173,NaN,NaN
1.551421e+09,34.743706,37.282718,36.729523,NaN,36.071935,31.545508,39.244258,34.791920,27.670525,37.856298,40.562759,NaN,32.596429,40.046824,34.751048,...,37.286573,32.539932,31.365265,30.417444,42.713888,35.687719,32.610254,NaN,NaN,34.474472,NaN,NaN,37.286859,38.009808,NaN
1.551422e+09,35.198855,38.011871,36.333911,35.488701,35.613133,31.223235,37.757226,32.528805,25.905566,37.424585,38.259191,NaN,32.647351,38.736527,31.741832,...,36.560167,32.628594,31.891656,31.509045,44.083700,35.730590,32.647672,NaN,33.636132,30.993492,NaN,30.505382,38.871570,38.150128,27.135176
1.551423e+09,35.753885,39.515138,35.683941,35.654141,36.317611,30.819836,36.971192,32.867365,25.571772,37.952419,36.544864,33.507180,31.973629,38.064815,29.550972,...,36.655475,31.577425,32.837938,31.872184,41.391144,34.298342,35.377237,NaN,33.303644,28.303780,NaN,30.645780,38.878773,39.714068,29.413497
1.551424e+09,36.035648,39.705286,35.553426,36.302840,37.296786,29.646012,37.274344,34.481275,25.656125,38.771818,35.803762,33.131247,31.646226,37.953408,29.145283,...,36.282938,29.995966,33.730365,31.459479,38.044115,33.085353,38.609206,NaN,31.862805,26.866304,31.821528,30.822450,38.332764,41.073771,30.116905


In [4]:
forecast_df = pd.read_csv('forecast_result_tainan.csv', index_col='time')
display(forecast_df.head())

,L05201_x,L09870,L01543,L06286,L01144,L05107,L05042,L01294,L00190,L02284,L01152,L00082,L01693,L03587,L01312,...,L05811,L06913,L04241,L05145,L04304,L01556,L00500,L98066,L07416,L04152,L03382,L04017,L04294,L05893,L05201_y
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-03-01 06:00:00,35.134963,40.085713,NaN,NaN,37.840989,NaN,40.093677,39.205644,30.599845,39.383541,NaN,NaN,NaN,NaN,NaN,...,NaN,31.488935,31.642293,29.726891,35.321547,NaN,35.607722,NaN,NaN,NaN,NaN,NaN,33.862173,NaN,NaN
2019-03-01 06:15:00,34.743706,37.282718,36.729523,NaN,36.071935,31.545508,39.244258,34.791920,27.670525,37.856298,40.562759,NaN,32.596429,40.046824,34.751048,...,37.286573,32.539932,31.365265,30.417444,42.713888,35.687719,32.610254,NaN,NaN,34.474472,NaN,NaN,37.286859,38.009808,NaN
2019-03-01 06:30:00,35.198855,38.011871,36.333911,35.488701,35.613133,31.223235,37.757226,32.528805,25.905566,37.424585,38.259191,NaN,32.647351,38.736527,31.741832,...,36.560167,32.628594,31.891656,31.509045,44.083700,35.730590,32.647672,NaN,33.636132,30.993492,NaN,30.505382,38.871570,38.150128,27.135176
2019-03-01 06:45:00,35.753885,39.515138,35.683941,35.654141,36.317611,30.819836,36.971192,32.867365,25.571772,37.952419,36.544864,33.507180,31.973629,38.064815,29.550972,...,36.655475,31.577425,32.837938,31.872184,41.391144,34.298342,35.377237,NaN,33.303644,28.303780,NaN,30.645780,38.878773,39.714068,29.413497
2019-03-01 07:00:00,36.035648,39.705286,35.553426,36.302840,37.296786,29.646012,37.274344,34.481275,25.656125,38.771818,35.803762,33.131247,31.646226,37.953408,29.145283,...,36.282938,29.995966,33.730365,31.459479,38.044115,33.085353,38.609206,NaN,31.862805,26.866304,31.821528,30.822450,38.332764,41.073771,30.116905


In [9]:
import datetime
date = datetime.datetime.fromtimestamp(1551420000)
print(date)

2019-03-01 14:00:00


In [11]:
forecast_df.index = pd.to_datetime(forecast_df.index, unit='s')
display(forecast_df.head())

,L04891,L09870,L01543,L06286,L01144,L05107,L05042,L01294,L00190,L02284,L01152,L00082,L01693,L03587,L01312,...,L05811,L06913,L04241,L05145,L04304,L01556,L00500,L98066,L07416,L04152,L03382,L04017,L04294,L05893,L05201
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-03-01 06:00:00,35.134963,40.085713,NaN,NaN,37.840989,NaN,40.093677,39.205644,30.599845,39.383541,NaN,NaN,NaN,NaN,NaN,...,NaN,31.488935,31.642293,29.726891,35.321547,NaN,35.607722,NaN,NaN,NaN,NaN,NaN,33.862173,NaN,NaN
2019-03-01 06:15:00,34.743706,37.282718,36.729523,NaN,36.071935,31.545508,39.244258,34.791920,27.670525,37.856298,40.562759,NaN,32.596429,40.046824,34.751048,...,37.286573,32.539932,31.365265,30.417444,42.713888,35.687719,32.610254,NaN,NaN,34.474472,NaN,NaN,37.286859,38.009808,NaN
2019-03-01 06:30:00,35.198855,38.011871,36.333911,35.488701,35.613133,31.223235,37.757226,32.528805,25.905566,37.424585,38.259191,NaN,32.647351,38.736527,31.741832,...,36.560167,32.628594,31.891656,31.509045,44.083700,35.730590,32.647672,NaN,33.636132,30.993492,NaN,30.505382,38.871570,38.150128,27.135176
2019-03-01 06:45:00,35.753885,39.515138,35.683941,35.654141,36.317611,30.819836,36.971192,32.867365,25.571772,37.952419,36.544864,33.507180,31.973629,38.064815,29.550972,...,36.655475,31.577425,32.837938,31.872184,41.391144,34.298342,35.377237,NaN,33.303644,28.303780,NaN,30.645780,38.878773,39.714068,29.413497
2019-03-01 07:00:00,36.035648,39.705286,35.553426,36.302840,37.296786,29.646012,37.274344,34.481275,25.656125,38.771818,35.803762,33.131247,31.646226,37.953408,29.145283,...,36.282938,29.995966,33.730365,31.459479,38.044115,33.085353,38.609206,NaN,31.862805,26.866304,31.821528,30.822450,38.332764,41.073771,30.116905


In [5]:
sec_info = pd.read_csv('SECINFO.csv')
#display(sec_info.head())

In [6]:
sec_info = sec_info.dropna()

In [7]:

sec_cols = forecast_df.columns
sectid_ids_gt = list(sec_info['SECTORID'])
not_in_gt = 0
in_gt = 0
sec_cols_gt = list()
for sec in sec_cols:
    if sec not in sectid_ids_gt:
        not_in_gt += 1
        #display('In sectodID: ' + sec)
    else:
        in_gt +=1
        sec_cols_gt.append(sec)
        #display(sec)
        
display('In forecast but not in ground_truth: ' + str(not_in_gt))
display('In forecast and ground truth: ' +str(in_gt))

'In forecast but not in ground_truth: 247'

'In forecast and ground truth: 116'

In [8]:
sec_info = sec_info.set_index('SECTORID')


In [9]:
sec_info.loc['L01707']
# 經緯 latitude-longitude

Unnamed: 0                          183
SECTORNAME    安和路五段(安和路五段81巷>海佃路一段213巷)
STARTIC                           I0169
ENDIC                             I0170
LENGTH                              195
LANENO                                2
INTDIR                                0
GROUPID                               0
LANELENGTH                            7
MODEL                               十字型
CMSDESC                               0
STATUS                                y
TRAVELTIME                           10
CAPACITY                             10
START_X                         120.219
START_Y                         23.0649
END_X                           120.218
END_Y                           23.0666
Name: L01707, dtype: object

### Save forecast results to csv

In [28]:
sec_info.loc['L01144']

Unnamed: 0                113
SECTORNAME    中華東路一段(小東路>華興路)
STARTIC                 I0838
ENDIC                   I0114
LENGTH                    130
LANENO                      2
INTDIR                      0
GROUPID                     0
LANELENGTH                  7
MODEL                     十字型
CMSDESC                     0
STATUS                      y
TRAVELTIME                 10
CAPACITY                   10
START_X               120.235
START_Y               23.0008
END_X                 120.235
END_Y                 22.9984
Name: L01144, dtype: object

In [ ]:
time1_forecast['L01144'].values

In [ ]:
m = folium.Map([23.0649, 120.219], zoom_start=14)
m

In [ ]:
m = folium.Map([23.0649, 120.219],zoom_start=12) #, tiles='stamentoner'

myIcon = folium.CustomIcon('https://raw.githubusercontent.com/HanInfinity/iron2018_FoliumAndLeaflet/master/dist/icon/marker.png',
                          icon_size = (30, 30),
                          icon_anchor = (15, 30))


for segment in time1_forecast.columns:
    forecast_value = time1_forecast[segment].values
    
    points1 = [sec_info.loc[segment]['START_Y'],sec_info.loc[segment]['START_X'] ]
    points2 = [sec_info.loc[segment]['END_Y'],sec_info.loc[segment]['END_X'] ]
    dist = distance(points1[0], points1[1], points2[0], points2[1])

    if 0 in points1:
        continue
        
    if 0 in points2:
        continue
    if dist > 100:
        continue
    #folium.Marker(points1, icon= myIcon, popup='預測速度: ' + str(forecast_value[0])).add_to(m)
    folium.features.ColorLine(
        [points1, points2],
        colors=[0],
        colormap=[section_color(forecast_value), section_color(forecast_value)],
        weight=8, opacity=1).add_to(m)
    



In [45]:
for a,b in timedict.items():
    print(a,b)

730 2019-05-05 07:30:00
800 2019-05-05 08:00:00
830 2019-05-05 08:30:00
900 2019-05-05 09:00:00
1200 2019-05-05 12:00:00
1230 2019-05-05 12:30:00
1300 2019-05-05 13:00:00
1330 2019-05-05 13:30:00
1900 2019-05-05 19:00:00
1930 2019-05-05 19:30:00
2000 2019-05-05 20:00:00
2030 2019-05-05 20:30:00


In [10]:
def add_colorline(points1, points2, forecast_value, m):
    folium.features.ColorLine(
            [points1, points2],
            colors=[0],
            colormap=[section_color(forecast_value), section_color(forecast_value)],
            weight=8, opacity=1).add_to(m)
    return m

In [32]:
forecast_df_time = forecast_df.loc['2019-05-05 07:30:00'][sec_cols_gt].to_frame().transpose()

In [33]:
forecast_df_time

,L01144,L05107,L05042,L01294,L02284,L01152,L01693,L01312,L00796,L04690,L00466,L04720,L03266,L03374,L04624,...,L03376,L03663,L03321,L00472,L01665,L00832,L01624,L03674,L01844,L03750,L04241,L04304,L01556,L04152,L03382
2019-05-05 07:30:00,36.092345,28.378919,40.928164,35.832162,41.180794,34.923623,34.836766,31.414859,31.704887,27.077374,35.542467,32.670106,34.588758,33.700631,33.465372,...,32.687026,30.378362,37.659515,31.869972,34.475896,44.539073,37.266615,34.292436,35.168533,31.196255,36.327147,32.77892,34.993195,30.228854,31.240842


In [42]:
for name, time in timedict.items():
    
    print('Processing time ', time)
    m = folium.Map([23.017100, 120.201147],zoom_start=12) #, tiles='stamentoner'

    forecast_df_time = forecast_df.loc[time][sec_cols_gt].to_frame().transpose()

    for segment in forecast_df_time.columns:
        forecast_value = forecast_df_time[segment].values
    
        points1 = [sec_info.loc[segment]['START_Y'],sec_info.loc[segment]['START_X'] ]
        points2 = [sec_info.loc[segment]['END_Y'],sec_info.loc[segment]['END_X'] ]
        dist = distance(points1[0], points1[1], points2[0], points2[1])

        if 0 in points1:
            continue

        if 0 in points2:
            continue
        if dist > 100:
            continue
        
    
        m = add_colorline(points1, points2, forecast_value, m)
    
    m.save(name + '.html')

Processing time  2019-05-05 07:30:00
Processing time  2019-05-05 08:00:00
Processing time  2019-05-05 08:30:00
Processing time  2019-05-05 09:00:00
Processing time  2019-05-05 12:00:00
Processing time  2019-05-05 12:30:00
Processing time  2019-05-05 13:00:00
Processing time  2019-05-05 13:30:00
Processing time  2019-05-05 19:00:00
Processing time  2019-05-05 19:30:00
Processing time  2019-05-05 20:00:00
Processing time  2019-05-05 20:30:00


In [ ]:
m.save('2019-05-05 12:00:00.html')


In [ ]:
START_X	START_Y	END_X	END_Y

In [ ]:
forecast_df['L01144'].describe()

15:43 帥到警察開罰單 尖峰時刻到緩和
15:43 帥到警察開罰單 平常低運量時刻
15:43 帥到警察開罰單 晚上尖峰時刻到緩和

In [12]:
time1 = '2019-05-05 12:00:00'
time2 = '2019-05-05 15:00:00'
time3 = '2019-05-05 18:00:00'
time4 = '2019-05-05 21:00:00'
time730 = '2019-05-05 07:30:00'
time800 = '2019-05-05 08:00:00'
time830 = '2019-05-05 08:30:00'
time900 = '2019-05-05 09:00:00'
time1200 = '2019-05-05 12:00:00'
time1230 = '2019-05-05 12:30:00'
time1300 = '2019-05-05 13:00:00'
time1330 = '2019-05-05 13:30:00'
time1900 = '2019-05-05 19:00:00'
time1930 = '2019-05-05 19:30:00'
time2000 = '2019-05-05 20:00:00'
time2030 = '2019-05-05 20:30:00'

In [32]:
a = time1_forecast.index

array([datetime.time(12, 0)], dtype=object)

In [31]:
time1_forecast = forecast_df.loc[time1][sec_cols_gt]
time2_forecast = forecast_df.loc[time2][sec_cols_gt]
time3_forecast = forecast_df.loc[time3][sec_cols_gt]
time4_forecast = forecast_df.loc[time4][sec_cols_gt]
time730_forecast = forecast_df.loc[time730][sec_cols_gt]
time800_forecast = forecast_df.loc[time800][sec_cols_gt]
time830_forecast = forecast_df.loc[time830][sec_cols_gt]
time900_forecast = forecast_df.loc[time900][sec_cols_gt]
time1200_forecast = forecast_df.loc[time1200][sec_cols_gt]
time1230_forecast = forecast_df.loc[time1230][sec_cols_gt]
time1300_forecast = forecast_df.loc[time1300][sec_cols_gt]
time1330_forecast = forecast_df.loc[time1330][sec_cols_gt]
time1900_forecast = forecast_df.loc[time1900][sec_cols_gt]
time1930_forecast = forecast_df.loc[time1930][sec_cols_gt]
time2000_forecast = forecast_df.loc[time2000][sec_cols_gt]
time2030_forecast = forecast_df.loc[time2030][sec_cols_gt]

In [ ]:
## Find statistics of forecast_df

forecast_df.describe()

In [41]:
def section_color(value):
    if value < 28:
        return 'red'
    elif value < 32:
        return 'orange'
    elif value <36:
        return 'green'
    else:
        return 'blue'

In [35]:
from math import radians, sin, cos, acos


def distance(slat, slon, elat, elon):
    return 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))


In [13]:
timedict = {'730': time730, 
'800': time800,
'830': time830,
'900': time900,
'1200': time1200,
'1230': time1230,
'1300': time1300,
'1330': time1330,
'1900': time1900,
'1930': time1930,
'2000': time2000,
'2030': time2030}

In [39]:
timedict.keys()

dict_keys(['730', '800', '830', '900', '1200', '1230', '1300', '1330', '1900', '1930', '2000', '2030'])

SyntaxError: invalid syntax (<ipython-input-40-f57ed4a26692>, line 8)